In [1]:
import os
import base64
import json
from pathlib import Path
from typing import List, Dict, Union
from html.parser import HTMLParser
from html import escape, unescape

In [2]:
class ImageEmbedder(HTMLParser):
    def __init__(self, html_path: Path):
        super().__init__()
        self.html_path = html_path
        self.result = ""
        self.success = []
        self.fail = []

    def handle_starttag(self, tag, attrs):
        if tag.lower() == "img":
            attrs_dict = dict(attrs)
            src = attrs_dict.get("src", "")
            img_path = (self.html_path.parent / src).resolve()
            if img_path.exists():
                try:
                    with open(img_path, "rb") as img_file:
                        encoded = base64.b64encode(img_file.read()).decode('utf-8')
                        mime_type = f"image/{img_path.suffix[1:]}"
                        new_src = f"data:{mime_type};base64,{encoded}"
                        attrs_dict["src"] = new_src
                        self.success.append(str(img_path))
                except Exception:
                    self.fail.append(str(img_path))
            else:
                self.fail.append(str(img_path))

            attrs_str = " ".join(f'{k}="{escape(v)}"' for k, v in attrs_dict.items())
            self.result += f"<img {attrs_str}>"
        else:
            attrs_str = " ".join(f'{k}="{escape(v)}"' for k, v in attrs)
            self.result += f"<{tag} {attrs_str}>" if attrs_str else f"<{tag}>"

    def handle_endtag(self, tag):
        self.result += f"</{tag}>"

    def handle_data(self, data):
        self.result += data

    def handle_entityref(self, name):
        self.result += f"&{name};"

    def handle_charref(self, name):
        self.result += f"&#{name};"


In [3]:
class HTMLBatchProcessor:
    def __init__(self, input_paths: List[Union[str, Path]], output_dir: Union[str, Path]):
        self.input_paths = [Path(p) for p in input_paths]
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.results = {
            "success": {},
            "fail": {}
        }

    def get_html_files(self) -> List[Path]:
        html_files = []
        for path in self.input_paths:
            if path.is_file() and path.suffix == ".html":
                html_files.append(path)
            elif path.is_dir():
                html_files.extend(path.rglob("*.html"))  #Explora subdirectorios también
        return html_files

    def process(self) -> Dict[str, Dict[str, List[str]]]:
        html_files = self.get_html_files()
        print(f"Se encontraron {len(html_files)} archivos HTML")

        for html_file in html_files:
            with open(html_file, "r", encoding="utf-8") as f:
                content = f.read()

            parser = ImageEmbedder(html_file)
            parser.feed(content)

            # Guardar nuevo archivo
            output_file = self.output_dir / html_file.name
            with open(output_file, "w", encoding="utf-8") as f_out:
                f_out.write(parser.result)

            self.results["success"][str(html_file)] = parser.success
            self.results["fail"][str(html_file)] = parser.fail

        return self.results

In [4]:

# Ejecución
if __name__ == "__main__":
    rutas = ["/content/drive/MyDrive/Prueba_tecnica_tuya/htmls_prueba"]
    salida = "/content/drive/MyDrive/Prueba_tecnica_tuya/html_modificados"

    processor = HTMLBatchProcessor(rutas, salida)
    resultado = processor.process()

    total_ok = sum(len(v) for v in resultado["success"].values())
    total_fail = sum(len(v) for v in resultado["fail"].values())
    print(f"\n Imágenes procesadas correctamente: {total_ok}")
    print(f"Imágenes con error: {total_fail}")

    resumen_path = "/content/drive/MyDrive/Prueba_tecnica_tuya/resumen_resultado.json"
    with open(resumen_path, "w", encoding="utf-8") as f:
        json.dump(resultado, f, indent=4, ensure_ascii=False)
    print(f"\n Resumen guardado en: {resumen_path}")

Se encontraron 3 archivos HTML

 Imágenes procesadas correctamente: 9
Imágenes con error: 0

 Resumen guardado en: /content/drive/MyDrive/Prueba_tecnica_tuya/resumen_resultado.json


In [7]:
readme_content = """# Prueba Técnica - Tuya

Este repositorio contiene la solución al primer punto de la prueba técnica de Tuya, donde se procesan archivos HTML para reemplazar imágenes por versiones embebidas en formato base64.

## Objetivo

Automatizar el procesamiento de archivos HTML para:

- Buscar imágenes dentro del código HTML.
- Convertirlas a base64.
- Reemplazarlas directamente en el HTML.
- Guardar el HTML modificado y un resumen en formato JSON con las imágenes procesadas correctamente o con error.

## Tecnologías usadas

- Python 3
- Librerías estándar: `os`, `base64`, `json`, `pathlib`, `html.parser`

## Estructura del proyecto

- `htmls_prueba/`: Carpeta de entrada con los archivos HTML originales.
- `html_modificados/`: Carpeta de salida con los archivos HTML modificados.
- `resumen_resultado.json`: Archivo con el resumen del procesamiento.

## Cómo ejecutar

1. Asegúrate de tener tus archivos HTML en la ruta especificada.
2. Ejecuta el script en Google Colab o un entorno local de Python.
3. Verifica el contenido de la carpeta de salida y el archivo `resumen_resultado.json`.

"""

# Ruta donde guardar el README
ruta_readme = "/content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imágenes/README.md"

# Guardar el archivo
with open(ruta_readme, "w", encoding="utf-8") as f:
    f.write(readme_content)

print(f"README.md creado en: {ruta_readme}")



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imágenes/README.md'

In [8]:
gitignore_content = """
# Ignorar cachés de Python
__pycache__/
*.py[cod]

# Ignorar checkpoints de Jupyter
.ipynb_checkpoints/

# Ignorar archivos temporales
*.swp
*.tmp

# Ignorar archivos ocultos del sistema
.DS_Store
Thumbs.db
"""

ruta_gitignore = "/content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imágenes/.gitignore"

with open(ruta_gitignore, "w", encoding="utf-8") as f:
    f.write(gitignore_content)

print(f".gitignore creado en: {ruta_gitignore}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imágenes/.gitignore'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/Prueba_tecnica_tuya/Ejercicio_1_HTML

In [11]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [12]:
!git remote add origin https://github.com/ddoncell94/tuya-technical-assessment.git

In [13]:
!git add .

error: open("drive/MyDrive/Documento sin título.gdoc"): Operation not supported
error: unable to index file 'drive/MyDrive/Documento sin título.gdoc'
fatal: adding files failed


In [14]:
!git commit -m "Subir archivos desde Colab"

On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git push -u origin master

In [10]:
from getpass import getpass
import os

# Configurar tu nombre de usuario y correo electrónico para Git
!git config --global user.name "ddoncell94"  # Reemplaza con tu nombre de usuario de GitHub
!git config --global user.email "danieladoncell@gmail.com"  # Reemplaza con tu correo de GitHub

# Solicitar el token de acceso personal de GitHub de manera segura
token = getpass("Introduce tu token de acceso personal de GitHub: ")

# Cambiar la URL remota para usar el token de acceso personal
!git remote set-url origin https://{token}@github.com/ddoncell94/tuya-technical-assessment.git

# Verificar que la URL remota esté configurada correctamente
!git remote -v

# Agregar todos los cambios al repositorio
!git add .

# Hacer commit de los archivos agregados
!git commit -m "Subir archivos desde Colab"

# Realizar el push al repositorio remoto
!git push -u origin master


Introduce tu token de acceso personal de GitHub: ··········
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
